# Reinvent4を使いEGFR kinase阻害剤様の構造生成モデルを作成する

転移学習とSamplingを利用してEGFR kinase阻害剤様の構造生成モデルを作成しましょう。

注）reinvent4は以下のようにCLIで利用しますが、本ハンズオンではjupyter上で実行するためsubprocessを利用します。

```bash
$ reinvent -l log.txt config.toml
```

## 転移学習に利用するEGFR kinase阻害剤の構造を確認する

rdkitを利用して構造を描画し確認します。

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
import pandas as pd
df = pd.read_csv('../data/genai/ChEMBL_EGFR.csv')
df.head(2)
Draw.MolsToGridImage([Chem.MolFromSmiles(smi) for smi in df.smiles][:20], molsPerRow=5)

## 設定ファイル(toml)を編集する

../data/genai/ 以下にTomlファイルがありますので、各自の環境に応じて編集をしてください。
以下に今回の転移学習で利用するファイル(transfer_learning.toml)を出力しますが、書き換える必要のある変数は

- device: cudaの場合はcuda:0, MacOS(M1,M2,M3)のgpuの場合はmps, gpuを利用しない場合はcpuを指定してください
- input_model_file, smiles_file, output_model_file, validation_smiles_fileは書き換える必要があります
- 今回はnum_epocを300にしていますがCPUのみの場合は学習に時間がかかるかもしれません。その場合は100に変更してください

In [ ]:
!cat ../data/genai/transfer_learning.toml

## 転移学習を実施
reinventコマンドは各自の環境に依存しますので設定してください

注）仮想環境のpathを調べたい場合は以下のコマンドを実行してください

```bash
$ % conda info --envs
```

In [ ]:
import subprocess

In [ ]:
subprocess.call(
    ["/home/iwatobipen/miniforge3/envs/reinvent4/bin/reinvent", "-l", "TL.log", "../data/genai/transfer_learning.toml"]
)

## tensroboardでログを確認
tensorboardはReinvent4と同じ環境下にインストールされているので、以下のコマンドを実行するとwebブラウザから転移学習の結果を確認することができます。

In [ ]:
!/home/iwatobipen/miniforge3/envs/reinvent4/bin/tensorboard --logdir tb_TL/

##　訓練前後のモデルでサンプリング（構造生成）

ここでは、転移学習前後の生成モデルから構造生成を行い、EGFR阻害剤様の構造が出力されているか確認します。
転移学習と同様にtomlファイルを設定する必要があります。

sampling.tomlは転移学習前の生成モデルからのサンプリングで、TL_sampling.tomlは転移学習後の生成モデルからのサンプリング用の設定ファイルです。
編集が必要な箇所はdevice,model_fileです。


In [ ]:
!cat ../data/genai/sampling.toml

In [ ]:
#訓練前のモデル
subprocess.call(
    ["/home/iwatobipen/miniforge3/envs/reinvent4/bin/reinvent", "-l", "sampling.log", "../data/genai/sampling.toml"]
)

In [ ]:
#訓練後のモデル
subprocess.call(
    ["/home/iwatobipen/miniforge3/envs/reinvent4/bin/reinvent", "-l", "TL_sampling.log", "../data/genai/TL_sampling.toml"]
)

## 構造を確認する

転移学習前後で生成モデルの出力する構造を確認します。転移学習後の生成モデルからはキナゾリン骨格が多く生成されていることを確認してください。

In [ ]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import PandasTools

In [ ]:
df1 = pd.read_csv('sampling.csv')
print(df1.shape)
df1.tail(2)

In [ ]:
PandasTools.AddMoleculeColumnToFrame(df1, smilesCol='SMILES')
Draw.MolsToGridImage(df1.ROMol[:20], molsPerRow=5)

In [ ]:
df2= pd.read_csv('TL_sampling.csv')
print(df2.shape)
df2.tail(2)

In [ ]:
PandasTools.AddMoleculeColumnToFrame(df2, smilesCol='SMILES')
Draw.MolsToGridImage(df2.ROMol[:20], molsPerRow=5)